# Prep

In [1]:
import pandas as pd
import numpy as np

## Functions

### Extract

In [6]:
def get_workouts(fp):
  all_workouts = []
  counter = 0

  with open(fp, encoding='utf-8') as f:
    clean_strings = [line.strip() for line in f.readlines()]
    no_empty_lines = list(filter(None, clean_strings))
    no_empty_lines.pop(0) # tirar a primeira linha, que é vazia

    single_workout = []
    for line in no_empty_lines:
      if 'Week' in line:
        if counter > 0:
          all_workouts.append(single_workout)
          single_workout = []
        counter += 1
      single_workout.append(line)

  return all_workouts

# Extract

In [18]:
raw_workouts = get_workouts(r'G:\My Drive\Projetos\workouts\2024_08_25 Workouts.csv')
raw_workouts.reverse()

In [25]:
raw_workouts[0]

['"Day 2 - Costas Bíceps Triceps · Day 2 · Week 2 · Padrão · Deload";"2023-10-03 14:24 h";"1:07 hr"',
 '"1. Lat Pulldowns with Wide Overhand Grip · Cable · 12 reps"',
 '#;KG;REPS',
 '1;55;15',
 '2;55;9',
 '3;55;8',
 '"2. Chest-Supported Low Rows with Neutral Grip · Machine · 12 reps"',
 '#;KG;REPS',
 '1;76;15',
 '2;80;15',
 '3;84;12',
 '"3. Triceps Pushdowns · Cable · 12 reps"',
 '#;KG;REPS',
 '1;35;12',
 '2;35;12',
 '3;35;8',
 '"4. Curls · EZ bar · 12 reps"',
 '#;KG;REPS',
 '1;16;15',
 '2;18;9',
 '3;18;8',
 '"5. Bent-Over Rows · Barbell · 12 reps"',
 '#;KG;REPS',
 '1;28;15',
 '2;30;12',
 '3;30;12',
 '"6. Upright Rows · Barbell · 12 reps"',
 '#;KG;REPS',
 '1;14;12',
 '2;14;12',
 '3;14;12',
 '"7. Lat Pulldowns with Close Neutral Grip · Cable · 10 reps"',
 '#;KG;REPS',
 '1;40;15',
 '2;45;11',
 '3;45;10']

# Transform